In [1]:
#!pip install boto3 minio


In [2]:
import boto3
from botocore.client import Config

# MinIO configuration
minio_endpoint = "http://minio:9000"  # or the IP of your MinIO instance
minio_root_user = "minio"  # Replace with your MINIO_ROOT_USER
minio_root_password = "minio123"  # Replace with your MINIO_ROOT_PASSWORD
bucket_name = "bronze"  # Replace with your bucket name

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=minio_root_user,
    aws_secret_access_key=minio_root_password,
    config=Config(signature_version='s3v4'),
    region_name="us-east-1"  # Region is required by boto3 but is arbitrary for MinIO
)


In [3]:
# List all buckets
buckets = s3_client.list_buckets()
for bucket in buckets['Buckets']:
    print(f'Bucket: {bucket["Name"]}')

# List objects in a specific bucket
objects = s3_client.list_objects_v2(Bucket=bucket_name)
for obj in objects.get('Contents', []):
    print(f'Object: {obj["Key"]}')


In [4]:
from minio import Minio

# Initialize MinIO client
minio_client = Minio(
    "minio:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False  # Set to True if you're using HTTPS
)

# List objects in a bucket
objects = minio_client.list_objects(bucket_name)
for obj in objects:
    print(obj.object_name)


In [5]:
import requests
import json
from datetime import datetime

# MinIO configuration the same as above
object_name = f"breweries_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"  # Unique object name based on timestamp

# Ensure the bucket exists
if not minio_client.bucket_exists(bucket_name):
    minio_client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")

# Fetch data from the API
response = requests.get("https://api.openbrewerydb.org/breweries")
breweries_data = response.json()

# Save JSON data to a file
json_data = json.dumps(breweries_data, indent=2)
json_file_path = "/tmp/breweries.json"
with open(json_file_path, 'w') as json_file:
    json_file.write(json_data)
    print(f"Data saved to {json_file_path}")

# Upload the JSON file to MinIO
minio_client.fput_object(bucket_name, object_name, json_file_path)
print(f"File '{object_name}' uploaded to MinIO bucket '{bucket_name}' successfully.")


In [6]:
# List objects in the bucket to find the CSV and JSON files
objects = minio_client.list_objects(bucket_name)
files = [obj.object_name for obj in objects]


In [7]:
from io import BytesIO
import pandas as pd

In [8]:
# Read and display CSV file
csv_file = [file for file in files if file.endswith('.csv')]
if csv_file:
    csv_file = csv_file[0]  # Assuming there's only one CSV file
    response = minio_client.get_object(bucket_name, csv_file)
    csv_data = pd.read_csv(BytesIO(response.read()))
    print(f"CSV Data from {csv_file}:")
    display(csv_data)

In [9]:
# Read and display JSON file
json_file = [file for file in files if file.endswith('.json')]
if json_file:
    json_file = json_file[0]  # Assuming there's only one JSON file
    response = minio_client.get_object(bucket_name, json_file)
    json_data = json.load(BytesIO(response.read()))
    print(f"JSON Data from {json_file}:")
    display(pd.DataFrame(json_data))